In [1]:
!pip install vaderSentiment

     |████████████████████████████████| 133kB 6.7MB/s 


In [2]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

In [3]:
text='The product is very very good'
vs.polarity_scores(text)

{'compound': 0.5379, 'neg': 0.0, 'neu': 0.59, 'pos': 0.41}

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

In [5]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

len(stopword_list)

177

In [8]:
# remove html tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

# html_tag('<html>jdhgcsdc</html>')?

In [9]:
# expand contractions
!pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

# con("Y'all can't expand")

     |████████████████████████████████| 327kB 5.3MB/s 
     |████████████████████████████████| 266kB 10.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.1-cp37-cp37m-linux_x86_64.whl size=85267 sha256=d468f6982b366ab7c981336dc7f48fc86a06f95c7d729aa9625874c64bd49924
  Stored in directory: /root/.cache/pip/wheels/e4/ab/f7/cb39270df8f6126f3dd4c33d302357167086db460968cfc80c
Successfully built pyahocorasick


In [10]:
# removal of special character
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text

# remove_sp("hello this is shashwat !!! wdicwb ^ *^$@ 123")

In [11]:
# remove stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

def remove_stop_words(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text

remove_stop_words("the is are and event not are stop words")

'event not stop words'

In [12]:
# 1: lower case
# 2: html tag
# 3: contraction
# 4: tokenize
# 5: stop words
df.tweet=df.tweet.apply(lambda x:x.lower())
df.tweet=df.tweet.apply(html_tag)
df.tweet=df.tweet.apply(con)
df.tweet=df.tweet.apply(remove_sp)
df.tweet=df.tweet.apply(remove_stop_words)

df.head()

,id,label,tweet
0,1,0,user father dysfunctional selfish drags kids d...
1,2,0,user user thanks lyft credit not use not offer...
2,3,0,bihday majesty
3,4,0,model love take time ur
4,5,0,factsguide society motivation


In [13]:
# df['compound']=df['news_article'].apply(lambda x: vs.polarity_scores(x)['compound'])
df['compound']=df['tweet'].apply(lambda x: vs.polarity_scores(x)['compound'])

In [14]:
conditions = [
    (df['compound'] >= 0.05),
    (df['compound'] > -0.05) & (df['compound'] < 0.05),
    (df['compound'] <= -0.05)
    ]

# create a list of the values we want to assign for each condition
values = ['Positive', 'Neutral', 'Negative']

# create a new column and use np.select to assign values to it using our lists as arguments
df['tier'] = np.select(conditions, values)

# display updated DataFrame
df

,id,label,tweet,compound,tier
0,1,0,user father dysfunctional selfish drags kids d...,-0.7650,Negative
1,2,0,user user thanks lyft credit not use not offer...,0.6705,Positive
2,3,0,bihday majesty,0.0000,Neutral
3,4,0,model love take time ur,0.6369,Positive
4,5,0,factsguide society motivation,0.3400,Positive
...,...,...,...,...,...
31957,31958,0,ate user isz youuu,0.0000,Neutral
31958,31959,0,see nina turner airwaves trying wrap mantle ge...,0.4588,Positive
31959,31960,0,listening sad songs monday morning otw work sad,-0.7351,Negative
31960,31961,1,user sikh temple vandalised calgary wso condem...,-0.5106,Negative


In [15]:
df.head()

,id,label,tweet,compound,tier
0,1,0,user father dysfunctional selfish drags kids d...,-0.7650,Negative
1,2,0,user user thanks lyft credit not use not offer...,0.6705,Positive
2,3,0,bihday majesty,0.0000,Neutral
3,4,0,model love take time ur,0.6369,Positive
4,5,0,factsguide society motivation,0.3400,Positive


In [16]:
x = df.iloc[:,2].values
y = df.iloc[:,-1].values

In [17]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 0,stratify = y)

In [18]:
x_train.shape

(22373,)

In [19]:
np.unique(y_train,return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([ 4653,  6151, 11569]))

In [20]:
np.unique(y_test,return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([1995, 2636, 4958]))

In [21]:
# Pipeline = Combine above 2 steps
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
# Syntax : Pipeline([('Variable 1',Method 1()),('Variable 2',Method 2())])

In [22]:
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())]) 

In [23]:
text_model.fit(x_train,y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('model',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', d

In [24]:
y_pred = text_model.predict(x_test)
y_pred

array(['Negative', 'Negative', 'Neutral', ..., 'Positive', 'Positive',
       'Positive'], dtype=object)

In [25]:
np.unique(y_pred,return_counts=True)

(array(['Negative', 'Neutral', 'Positive'], dtype=object),
 array([1599, 3113, 4877]))

In [26]:
y_test

array(['Negative', 'Neutral', 'Neutral', ..., 'Positive', 'Positive',
       'Positive'], dtype=object)

In [27]:
# Evaluation
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [28]:
accuracy_score(y_pred,y_test)*100

84.56564813849202

In [29]:
confusion_matrix(y_pred,y_test)

array([[1315,   99,  185],
       [ 384, 2375,  354],
       [ 296,  162, 4419]])

In [30]:
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

    Negative       0.66      0.82      0.73      1599
     Neutral       0.90      0.76      0.83      3113
    Positive       0.89      0.91      0.90      4877

    accuracy                           0.85      9589
   macro avg       0.82      0.83      0.82      9589
weighted avg       0.86      0.85      0.85      9589



In [31]:
text_model.predict(["#people aren't protesting #trump because a #republican not every on e won never hw is dumbo kjkjwe @#$%^&*Â€Â¦ "])

array(['Negative'], dtype=object)

In [32]:
!pip install streamlit
!pip install pyngrok==4.1.1
from pyngrok import ngrok

     |████████████████████████████████| 7.5MB 6.6MB/s 
     |████████████████████████████████| 112kB 40.1MB/s 
     |████████████████████████████████| 81kB 7.7MB/s 
     |████████████████████████████████| 4.6MB 38.3MB/s 
     |████████████████████████████████| 163kB 39.4MB/s 
     |████████████████████████████████| 122kB 37.2MB/s 
     |████████████████████████████████| 71kB 7.4MB/s 
  Created wheel for blinker: filename=blinker-1.4-cp37-none-any.whl size=13448 sha256=866071c847bac2fd547b1134552629742281970296365baf9b4757fc7ec7eab1
  Stored in directory: /root/.cache/pip/wheels/92/a0/00/8690a57883956a301d91cf4ec999cc0b258b01e3f548f86e89
Successfully built blinker
ERROR: google-colab 1.0.0 has requirement ipykernel~=4.10, but you'll have ipykernel 5.5.0 which is incompatible.
  Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1


  Created wheel for pyngrok: filename=pyngrok-4.1.1-cp37-none-any.whl size=15971 sha256=1c4e7f70cae370beb309b48df9c3e636fdb405ec486631b1248d48ca7e56ea77
  Stored in directory: /root/.cache/pip/wheels/97/71/0d/1695f7c8815c0beb3b5d9b35d6eec9243c87e6070fbe3977fa
Successfully built pyngrok


In [36]:
%%writefile app.py
import streamlit as st
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os

import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')
stopword_list.remove('no')
stopword_list.remove('not')

# remove html tag
def html_tag(text):
  soup=BeautifulSoup(text,"html.parser")
  new_text=soup.get_text()
  return new_text

# !pip install contractions
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand

# removal of special character
import re
def remove_sp(text):
  pattern=r'[^A-Za-z0-9\s]'
  text=re.sub(pattern,'',text)
  return text


# remove stop words
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

def remove_stop_words(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text


st.title("Tweet's Sentiment Analysis")

df = pd.read_csv('/content/drive/MyDrive/train.csv')

df.tweet=df.tweet.apply(lambda x:x.lower())
df.tweet=df.tweet.apply(html_tag)
df.tweet=df.tweet.apply(con)
df.tweet=df.tweet.apply(remove_sp)
df.tweet=df.tweet.apply(remove_stop_words)

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
vs=SentimentIntensityAnalyzer()

df['compound']=df['tweet'].apply(lambda x: vs.polarity_scores(x)['compound'])

conditions = [
    (df['compound'] >= 0.05),
    (df['compound'] > -0.05) & (df['compound'] < 0.05),
    (df['compound'] <= -0.05)
    ]

# create a list of the values we want to assign for each condition
values = ['Positive', 'Neutral', 'Negative']

# create a new column and use np.select to assign values to it using our lists as arguments
df['tier'] = np.select(conditions, values)

x = df.iloc[:,2].values
y = df.iloc[:,-1].values

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
text_model = Pipeline([('tfidf',TfidfVectorizer()),('model',SVC())]) 
text_model.fit(x,y)
select = st.text_input('Enter your message')

if(st.markdown(
    '<span class="badge badge-pill badge-success"> Badge </span>',
    unsafe_allow_html=True
)):
  op = text_model.predict([select])
  ans=op[0]

  if ans == 'Positive':
    st.success("Positive 🙂")
  if ans == 'Negative':
    st.error("Negative 😠")
  if ans== 'Neutral':
    st.warning("Neutral 😐")

# st.title(op[0])

Overwriting app.py


In [34]:
!nohup streamlit run app.py &
url = ngrok.connect(port='8501')
url

nohup: appending output to 'nohup.out'


'http://131ee203ae73.ngrok.io'